In [2]:
import numpy as np

In [3]:
def PrettyPrintBinary(state):
    outstr = ""
    if (type(state) == list and type(state[0]) == tuple):
        for s in state:
            outstr += (str)(s[0])
            outstr += "|"
            outstr += s[1]
            outstr += "> "
        print(outstr)
    elif (type(state) == list and type(state[0]) == np.float64):
        sto = 0
        for s in range(len(state)):
            if (len(bin(s)) > sto):
                sto = len(bin(s))
        print(sto)
        for s in range(len(state)):
            if(state[s] != 0):
                outstr += str(state[s])
                outstr += "|"
                addstr = str(bin(s))[2:]
                if (len(addstr) < sto - 2):
                    
                outstr += str(bin(s))[2:]
                outstr += "> "
        print(outstr)
    else:
        print("State is not valid")
def PrettyPrintInteger(state):
    
    print("TODO")

IndentationError: expected an indented block after 'if' statement on line 21 (525198013.py, line 23)

In [31]:
class qubit:
    

    def __init__(self, state):
        
        if (type(state) == list and type(state[0]) == np.float64):
            self.__state = np.asarray(state)
        elif (type(state) == list and type(state[0]) == tuple):
            self.max = 0
            for s in state:
                if (int(s[1], 2) > self.max):
                    self.max = int(s[1], 2)
            self.max = self.max + 1
            self.__state = np.zeros(self.max)
            print(self.__state)
            for s in state:
                place = int(s[1], 2)
                self.__state[place] = s[0]
    
    def PrettyPrintBinary(self):
        outstr = ""
        
        l = len(bin(self.max))
        print(l)
        for s in range(len(self.__state)):
            if(self.__state[s] != 0):
                outstr += str(self.__state[s])
                outstr += "|"
                addstr = str(bin(s))[2:]
                # if (len(addstr) < l - 2):
                    
                outstr += str(bin(s))[2:]
                outstr += "> "
        print(outstr)    

    def myPrint(self):
        print(self.__state)

    


In [32]:
myState2=[
   (np.sqrt(0.1)*1.j, '101'),
   (np.sqrt(0.5), '000') ,
   (-np.sqrt(0.4), '010' )]
myEquivilentState2 = [np.sqrt(0.5), 0, -np.sqrt(0.4), 0, 0, np.sqrt(0.1)*1.j, 0, 0]
q = qubit(myState2)
q.myPrint()
q.PrettyPrintBinary()


[0. 0. 0. 0. 0. 0.]
[ 0.70710678  0.         -0.63245553  0.          0.          0.        ]
5
0.7071067811865476|0> -0.6324555320336759|10> 


C:\Users\Aidan\AppData\Local\Temp\ipykernel_16852\1607636143.py:18: ComplexWarning: Casting complex values to real discards the imaginary part
  self.__state[place] = s[0]
